In [1]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql import SparkSession
from custom_package.transformer import  DetectRegionsText
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
import os
import sys
import json
import re

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

c:\Users\hp\parser_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PipelineSession") \
    .master("local") \
    .config("spark.hadoop.fs.defaultFS","file:///")\
    .getOrCreate()

# Read images data from Spark
images_df = spark.read.format("binaryFile").option("pathGlobFilter", "*.*").load("../assets/cv_test_for_pipeline/")
# Convert Spark DataFrame to Pandas DataFrame
images_df_pandas = images_df.select("path").toPandas()
# Clean path column
images_df_pandas['path'] = images_df_pandas['path'].str.replace(r'^file:/', '', regex=True)
    
# Initialize transformer
detector = DetectRegionsText(inputCol="path", outputCol="json_res")

In [3]:
p = Pipeline(stages=[detector])
pm = p.fit(images_df_pandas)

In [4]:
pm.transform(images_df_pandas)

c:\Users\hp\parser_project\Lib\site-packages\ultralytics\nn\tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # 

Start Extract Process
End Extract Process
Start Generate Process
End Generate Process


DataFrame[path: string, path_of_image: string, predictions: array<map<string,bigint>>, text: string, json_res: string]

In [5]:
# transformed = pm.transform(images_df_pandas)

In [6]:
# transformed.show()

In [7]:
# transformed_df = transformed.toPandas()
# columns_to_check =['path','text']
# mask = ~transformed_df[columns_to_check].apply(lambda x: x.eq('NaN')).any(axis=1)
# final_df_cleaned = transformed_df[mask]
# final_df_cleaned = final_df_cleaned.drop(['path_of_image', 'predictions','text'] , axis=1)
# final_df_cleaned['name'] = final_df_cleaned['path'].apply(
#     lambda path: os.path.splitext(os.path.basename(path))[0]
# )
# final_df_cleaned['json_res'] = final_df_cleaned['json_res'].apply(
#     lambda res: re.sub(r'```', '', re.sub(r'json', '', res))
# )

In [8]:
# final_df_cleaned

In [9]:
warehouse_dir = spark.conf.get("spark.sql.warehouse.dir")
print(f"Spark Warehouse Directory: {warehouse_dir}")

Spark Warehouse Directory: file:/C:/Users/hp/parser_project/code/spark-warehouse


In [10]:
pm.save(warehouse_dir)

In [18]:
images_df = spark.read.format("binaryFile").option("pathGlobFilter", "*.*").load("../assets/cvpipeline/")
# Convert Spark DataFrame to Pandas DataFrame
images_df_pandas = images_df.select("path").toPandas()
# Clean path column
images_df_pandas['path'] = images_df_pandas['path'].str.replace(r'^file:/', '', regex=True) 

In [19]:
loaded_pipeline = PipelineModel.load(warehouse_dir)

In [20]:
transformed_data = loaded_pipeline.transform(images_df_pandas)

c:\Users\hp\parser_project\Lib\site-packages\ultralytics\nn\tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # 

Start Extract Process
End Extract Process
Start Generate Process
End Generate Process


In [21]:
transformed_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                path|       path_of_image|         predictions|                text|            json_res|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|c:/Users/hp/parse...|c:/Users/hp/parse...|                  []|Clients\n° ”\n\ B...|```json\n{\n  "no...|
|                 NaN|c:/Users/hp/parse...|[{class -> 3, bbo...|                 NaN|Veuillez fournir ...|
|                 NaN|c:/Users/hp/parse...|[{class -> 3, bbo...|                 NaN|Veuillez me fourn...|
|                 NaN|c:/Users/hp/parse...|[{class -> 3, bbo...|                 NaN|Veuillez fournir ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [22]:
transformed_df = transformed_data.toPandas()
columns_to_check =['path','text']
mask = ~transformed_df[columns_to_check].apply(lambda x: x.eq('NaN')).any(axis=1)
final_df_cleaned = transformed_df[mask]
final_df_cleaned = final_df_cleaned.drop(['path_of_image', 'predictions','text'] , axis=1)
final_df_cleaned['name'] = final_df_cleaned['path'].apply(
    lambda path: os.path.splitext(os.path.basename(path))[0]
)
final_df_cleaned['json_res'] = final_df_cleaned['json_res'].apply(
    lambda res: re.sub(r'```', '', re.sub(r'json', '', res))
)

In [23]:
final_df_cleaned

,path,json_res,name
0,c:/Users/hp/parser_project/assets/cvpipeline/C...,"\n{\n ""nom_prenom"": ""Hicham Rh"",\n ""profil"":...",CV Hicham Rhafoune 2024
